# Effectiveness - Add a single point

In [1]:
import sys
sys.path.append("../..")

from sklearn import svm
import numpy as np
from copy import deepcopy

import src

from src.data_utils import (
    load_npy, save_npy, preprocess_data, variance, normalize, generate_union_description
)

In [ ]:
# read
x_train, y_train, x_test, y_test, columns_name = preprocess_data('train_100p_3c.csv', 'test_100p_3c.csv')

# pick a point from test for add
select_idx = 25

add_point_x = x_test[select_idx][:]
add_point_y = y_test[select_idx]

x_train_added = np.append(x_train, [x_test[select_idx,:]], axis=0)
y_train_added = np.append(y_train, y_test[select_idx])

x_test_added = np.delete(x_test, select_idx, axis=0)
y_test_added = np.delete(y_test, select_idx)

x_test = x_test_added
y_test = y_test_added

union_d = generate_union_description(len(x_train), 10)
union_d_added = deepcopy(union_d)
union_d_added[4] = np.append(union_d_added[4], len(x_train))
union_d_added_once = deepcopy(union_d_added)
union_d_added[9] = np.append(union_d_added[9], len(x_train) + 1)

In [ ]:
#mc plus
model = svm.SVC(decision_function_shape='ovo')

mc_plus_ov_101 = src.mc_owen(x_train_added, y_train_added, x_test_added, y_test_added, model, union_d_added_once,
                                  1000*101, 25)
save_npy('mc_plus_ov_101.npy', mc_plus_ov_101)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [18, 56, 89]

for idx in idxs:
    new_union_d = deepcopy(union_d)
    for _ in new_union_d:
        if idx in _:
            _.remove(idx)
    save_npy('mc_plus_ov_99_del'+str(idx)+'.npy',
             src.mc_owen(np.delete(x_train, idx, axis=0), np.delete(y_train, idx),
                     x_test_added, y_test_added, model, new_union_d,
                              1000*99, 25)
            )
# mc part
model = svm.SVC(decision_function_shape='ovo')


for i in range(1):
    mc_ov_101 = src.mc_owen(x_train_added, y_train_added, x_test_added, y_test_added, model, union_d_added_once,
                                 101*100, 25)
    save_npy('mc_ov_101.npy'+str(i), mc_ov_101)
    # mc_ov_101 and mc_ov_101 (many permutations)
    var1 = variance(mc_plus_ov_101[:100], normalize(mc_plus_ov_101[:100], mc_ov_101[:100]))
    print("variance of MC   \t : %.10f" % var1)

# plus version
model = svm.SVC(decision_function_shape='ovo')

pivot_owen = src.PivotOwen(x_train, y_train, x_test, y_test, model, init_sv=[], union_description=union_d)
pivot_owen.prepare(100000, proc_num=25)

pivot_ov_101 = pivot_owen.add_single_point(add_point_x, add_point_y, 4,
                                           101*100)
save_npy('pivot_ov_101_plus.npy', pivot_ov_101)

# init sv
model = svm.SVC(decision_function_shape='ovo')

mc_plus_ov_100 = src.mc_owen(x_train, y_train, x_test_added, y_test_added, model, union_d,
                                 1000*len(y_train), 4)
save_npy('mc_plus_ov_100.npy', mc_plus_ov_100)

model = svm.SVC(decision_function_shape='ovo')
mc_ov_101 = src.mc_owen(x_train_added, y_train_added, x_test_added, y_test_added, model, union_d_added_once,
                           100*len(y_train_added), 1)
save_npy('mc_ov_101.npy', mc_ov_101)

# delta
init_ov = load_npy('mc_plus_ov_100.npy')

delta_owen = src.DeltaOwen(x_train, y_train, x_test_added, y_test_added, model,
                                init_ov, union_d)
delta_ov_101 = delta_owen.add_single_point(add_point_x, add_point_y, 4,
                            100*(len(y_train)+1))

save_npy('delta_ov_101.npy', delta_ov_101)

# pivot
init_ov = load_npy('mc_plus_ov_100.npy')

pivot_owen = src.PivotOwen(x_train, y_train, x_test_added, y_test_added, model,
                              init_ov, union_d)

lov_100 = pivot_owen.prepare(1000*len(y_train))
save_npy('lov_100.npy', lov_100)

ov_101 = pivot_owen.add_single_point(add_point_x[0], add_point_y[0], 4,
                            100*(len(y_train)+1), flags={'flag_lov': True})

pivot_owen.x_train = np.append(x_train, [add_point_x[0]], axis=0)
pivot_owen.y_train = np.append(y_train, add_point_y[0])
pivot_owen.union_description = union_d_added_once

pivot_ov_101 = pivot_owen.add_single_point(add_point_x[1], add_point_y[1], 9,
                                        100*(len(y_train)+2), flags={'flag_lov': True})

save_npy('pivot_ov_101.npy', pivot_ov_101)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_ov = load_npy('mc_plus_ov_100.npy')

# KNN
heur_owen = src.HeurOwen(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_ov, union_d, 
                              params={'method': 'knn'})
heur_owen.prepare(params={'n_neighbors': 3})
knn_ov_101 = heur_owen.add_single_point(add_point_x, add_point_y)
save_npy('knn_ov_101.npy', knn_ov_101)

# for simi
model = svm.SVC(decision_function_shape='ovo')

idxs = [18, 56, 89]
ovs = list()

for idx in idxs:
    new_union_d = deepcopy(union_d)
    for _ in new_union_d:
        if idx in _:
            _.remove(idx)
    ov = src.mc_owen(np.delete(x_train, idx, axis=0),
                          np.delete(y_train, idx),
                          x_test_added, y_test_added,
                          model, new_union_d, 1000*99, 4)
    save_npy('mc_plus_sv_99_del'+str(idx)+'.npy', ov)
    ovs.append(ov)

ovs = np.asarray(ovs)

save_npy('knn_plus_ovs.npy', ovs)


# KNN+
heur_shap = src.HeurOwen(x_train, y_train,
                              x_test_added, y_test_added,
                              model, init_ov, union_d,
                              params={'method': 'knn+'})

heur_shap.prepare(flags={'exact': False, 'train': False},
                  params={'n_neighbors': 3,
                          'simi_type': 'ed',
                          'f_shap': 'n*n',
                          'rela': ['poly', 1],
                          'train_idxs': [18, 56, 89],
                          'train_ovs': load_npy('knn_plus_ovs.npy')})

knn_plus_ov_101 = heur_shap.add_single_point(add_point_x, add_point_y, 4)
save_npy('knn_plus_ov_101.npy', knn_plus_ov_101)

In [ ]:
model = svm.SVC(decision_function_shape='ovo')
init_ov = load_npy('mc_plus_ov_100.npy')
base_owen = src.BaseOwen(x_train, y_train, x_test_added,
                              y_test_added, model, init_ov, union_d)

# average based
base_ov_101_gavg = base_owen.add_single_point(add_point_x,
                                            add_point_y,
                                            4,
                                            params={'method': 'gavg'})

# leave one out based
base_ov_101_lavg = base_owen.add_single_point(add_point_x,
                                            add_point_y,
                                            4,
                                            params={'method': 'lavg'})

save_npy('base_ov_101_gavg.npy', base_ov_101_gavg)

save_npy('base_ov_101_lavg.npy', base_ov_101_lavg)

# comparison
def comp(base_v, comp_ov, name):
    var = variance(base_v, normalize(base_v, comp_ov))
    print("variance of %s   \t : %.10f" % (name, var))

mc_plus_ov_101 = load_npy('mc_plus_ov_101.npy')

comp(mc_plus_ov_101, load_npy('delta_ov_101.npy'), 'delta')
comp(mc_plus_ov_101, load_npy('pivot_ov_101.npy'), 'pivot')
comp(mc_plus_ov_101, load_npy('knn_ov_101.npy'), 'knn')
comp(mc_plus_ov_101, load_npy('knn_plus_ov_101.npy'), 'knn+')
comp(mc_plus_ov_101, load_npy('mc_ov_101.npy'), 'mc')
comp(mc_plus_ov_101, load_npy('base_ov_101_lavg.npy'), 'base lavg')
comp(mc_plus_ov_101, load_npy('base_ov_101_gavg.npy'), 'base gavg')